In [1]:
import pandas as pd

In [2]:
import os
def data2df (path, label):
    file, text = [], []
    for f in os.listdir(path):
        file.append(f)
        fhr = open(path+f, 'r', encoding="utf8", errors='ignore') 
        t = fhr.read()
        text.append(t)
        fhr.close()
    return(pd.DataFrame({'file': file, 'text': text, 'class':label}))

dfnonpro = data2df('C:/Users/zengs/OneDrive/Desktop/Grad Term 2/CIS 509/Assignment 5/HealthProNonPro/NonPro/', 0) 
dfpro = data2df('C:/Users/zengs/OneDrive/Desktop/Grad Term 2/CIS 509/Assignment 5/HealthProNonPro/Pro/', 1) 

df = pd.concat([dfnonpro, dfpro], axis=0)
df.head()

,file,text,class
0,a1.txt,there is a good wikipedia article on this.,0
1,a10.txt,there is no cure for gout but it can be treate...,0
2,a100.txt,Laboratory rats whose diets are supplemented w...,0
3,a101.txt,Heres a simple set of exercises that I do dail...,0
4,a102.txt,How to restore 20/20 vision.\n\nExercise 1: \...,0


In [3]:
# setup the data
X, y = df['text'], df['class']
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=1)

In [4]:
import re
import string
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

def preprocess(text):
    # replace one or more white-space characters with a space
    regex = re.compile(r"\s+")                               
    text = regex.sub(' ', text)    
    # lower case
    text = text.lower()          
    # remove digits and punctuation
    regex = re.compile(r"[%s%s]" % (string.punctuation, string.digits))
    text = regex.sub(' ', text)           
    # remove stop words
    sw = stopwords.words('english')
    text = text.split()                                              
    text = ' '.join([w for w in text if w not in sw]) 
    # remove short words
    text = ' '.join([w for w in text.split() if len(w) >= 2])
    # stem words
    text = ' '.join([(PorterStemmer()).stem(w) for w in text.split()]) 
    # lemmatize
    text = ' '.join([(WordNetLemmatizer()).lemmatize(w) for w in text.split()]) 
    return text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zengs\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# setup the preprocessing->model pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
clf = Pipeline(steps=[
    ('pp', TfidfVectorizer(
        preprocessor=preprocess, 
        use_idf=True, smooth_idf=True, norm='l2',
        min_df=1, max_df=1.0, max_features=None, 
        ngram_range=(1, 1))),
    ('mdl', MultinomialNB())
    ])

In [6]:
# setup grid search
from sklearn.model_selection import GridSearchCV
param_grid = {
    'pp__norm':['l1','l2',None],
    'mdl__alpha':[0.01, 0.1, 0.2, 0.5, 1]
}
gscv = GridSearchCV(clf, param_grid, iid=False, cv=5, return_train_score=False)

In [7]:
# search for best estimator
gscv.fit(Xtrain, ytrain)
print(gscv.best_estimator_, "\n")

Pipeline(memory=None,
     steps=[('pp', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...se_idf=True, vocabulary=None)), ('mdl', MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True))]) 



In [8]:
# evaluate best_estimator_ on test data
ypred = gscv.best_estimator_.predict(Xtest)
from sklearn import metrics
print (metrics.accuracy_score(ytest, ypred))
print (metrics.confusion_matrix(ytest, ypred))
print (metrics.classification_report(ytest, ypred))

0.9444949954504095
[[495  56]
 [  5 543]]
              precision    recall  f1-score   support

           0       0.99      0.90      0.94       551
           1       0.91      0.99      0.95       548

   micro avg       0.94      0.94      0.94      1099
   macro avg       0.95      0.94      0.94      1099
weighted avg       0.95      0.94      0.94      1099

